In [4]:
from __future__ import print_function
from scipy.optimize import fsolve
import math
from itertools import chain
from itertools import islice
import autograd.numpy as np
import numdifftools as nd
from autograd.scipy.integrate import odeint
from autograd import elementwise_grad, grad, jacobian
from autograd.builtins import tuple
import matplotlib.pyplot as plt
import math
import csv
import sys

In [1]:
kb = 1.381e-23
h = 6.626e-34
R = 8.314

# Things to input: 

T = float(input('Enter Temperature (enter float) :'))
rno = int(input("Enter the number of reactions: "))
char = []
i = 0
print('Enter each species (enter "done" when you are done)')
#Example Input ---------------------------------------------
# Species 1:CO
# Species 2:CO*
# Species 3:O2
# Species 4:O*
# Species 5:CO2
# Species 6:done
#-----------------------------------------------------------
while True:
    i += 1
    spec = input(f"Species {i}:")
    if (spec == 'done'):
        break
    else:
        char.append(spec)
spec_no = len(char)
char += ['*']
print(char)

Enter Temperature (enter float) :300
Enter the number of reactions: 3
Enter each species (enter "done" when you are done)
Species 1:CO
Species 2:CO*
Species 3:O2
Species 4:O*
Species 5:CO2
Species 6:done
['CO', 'CO*', 'O2', 'O*', 'CO2', '*']


In [5]:
st_mat = np.zeros([rno, spec_no + 1]) # Plus 1 is for the catalytic site (*)
# st_mat stores the stoich. of each species for each reaction in a matrix
# Example input ------------------------------------------------------
# [[-1.  1.  0.  0.  0. -1.]
# [ 0.  0. -1.  2.  0. -2.]
# [ 0. -1.  0. -1.  1.  2.]]
# ---------------------------------------------------------------------
for j in range(rno):
    print("For reaction", j+1)
    for i in range(spec_no+1):
        st_spec = input(f"Enter the stoich. for {char[i]}: ")
        st_spec = int(st_spec)
        st_mat[j][i] = st_spec
        print(st_mat)

For reaction 1
Enter the stoich. for CO: -1
[[-1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
Enter the stoich. for CO*: 1
[[-1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
Enter the stoich. for O2: 0
[[-1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
Enter the stoich. for O*: 0
[[-1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
Enter the stoich. for CO2: 0
[[-1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
Enter the stoich. for *: -1
[[-1.  1.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
For reaction 2
Enter the stoich. for CO: 0
[[-1.  1.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
Enter the stoich. for CO*: 0
[[-1.  1.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
Enter the stoich. for O2: -1
[[-1.  1.  0.  0.  0. -1.]
 [ 0.  0. -1.  0.  0.  0.]
 [ 

In [6]:
interm_no = 0 # number of intermediates, initialized to 0
interm_index = [] # index of all intermediates, eg. if the species are
# [A, A*, B, B*, AB] then interm_index = [1,3]

for i in range(len(char)):
    # Find the number of intermediates based on the character intputs that you give
    ifstar = char[i].find('*')
    if ifstar > 0:
        interm_no += 1
        interm_index.append(i)

print('Please enter the index of the species which are products: ')
prod_index = [] # Follows the same convention as the interm_index
i = 0
while True:
    i += 1
    pd_in = input("Enter: ")
    if (pd_in == 'done'):
        break
    else:
        prod_index.append(int(pd_in))
        
# Example input --------------------------------------------
# Please enter the index of the species which are products: 
# Enter: 4
# Enter: done
#----------------------------------------------------------


prod_no = len(prod_index)
interm_index = np.array(interm_index)
print(f'There are a total of {interm_no} intermediates at the entries {interm_index+1}.')
ts_no = spec_no - interm_no - prod_no + 1

Please enter the index of the species which are products: 
Enter: 4
Enter: done
There are a total of 2 intermediates at the entries [2 4].


In [7]:
GTS = [] # Free energy of Transition States
GIS = [] # Free energy of Intermediates
GP = [] # Free energy of Products
G_char = [] # stores the string of all Gs

# Asks you to enter the Gibbs free energy for each intermediate, reactant and product
print('Enter the Gibbs Free Energy (eV) with reference to the reactants')
for i in range(ts_no):
    gt = float(input(f'G_TS{i+1}: '))
    GTS.append(gt)
    G_char.append(f'G_TS{i+1}: ')

for i in range(interm_no):
    gi = float(input(f'G_{char[interm_index[i]]}: '))
    GIS.append(gi)
    G_char.append(f'G_{char[interm_index[i]]}: ')

for i in range(prod_no):
    gp = float(input(f'G_{char[prod_index[i]]}: '))
    GP.append(gp)
    G_char.append(f'G_{char[prod_index[i]]}: ')

# Example input --------------------------------------------
# Enter the Gibbs Free Energy (eV) with reference to the reactants
# G_TS1: 0.0
# G_TS2: 0.62
# G_TS3: 0.72
# G_CO*: -1.2
# G_O*: -0.75
# G_CO2: -0.96
# -----------------------------------------------------------
    

Enter the Gibbs Free Energy (eV) with reference to the reactants
G_TS1: 0.0
G_TS2: 0.62
G_TS3: 0.72
G_CO*: -1.2
G_O*: -0.75
G_CO2: -0.96


In [8]:
G_dict = {'GTS':GTS, 'GIS':GIS, 'GP':GP} # A diction. that stores 
# all the Gs based on the category
dreact = {} # diction. that stores the species information like
# stoich. matrix and the pressures, coverages

Theta_guess = [] # initial guess of the coverages

for i in range(spec_no):
    if i in interm_index:
        tempC = float(input(f"Enter a guess coverage for {char[i]}: "))
        dreact[i] = [st_mat[:,i], tempC]
        Theta_guess.append(tempC)
    elif i in prod_index:
        dreact[i] = [st_mat[:,i],float(input(f"Enter a the pressure for {char[i]}: "))]
    else:
        dreact[i] = [st_mat[:,i],float(input(f"Enter a the pressure for {char[i]}: "))]

# Example input ---------------------------------------------
# Enter a the pressure for CO: 1.0
# Enter a guess coverage for CO*: 0.2
# Enter a the pressure for O2: 10.0
# Enter a guess coverage for O*: 0.5
# Enter a the pressure for CO2: 0.0
# -----------------------------------------------------------

Enter a the pressure for CO: 1.0
Enter a guess coverage for CO*: 0.2
Enter a the pressure for O2: 10.0
Enter a guess coverage for O*: 0.5
Enter a the pressure for CO2: 0.0


In [9]:
def kd(dGi):
    return (kb*T/h)*np.exp(-dGi/R/T)

def rate_constant(Gib):
    "This function returns a matrix of size (no. of reactions x 2)"
    "where the first column stores the forward rate constants and"
    "the second column stores the reverse rate constants"
    
    k_ent = np.zeros([rno,2]) # Each row represents every reaction and 
    # first column represents the forward reaction and the second represents 
    # the backward reaction
    
    G_full = [] # An array of rno*spec_no multiplies the GISs with it's correspodning stoich.
    
    for i in range(rno): # iterating over each reaction
        
        st_vec = st_mat[i] # stoich. vector of that particular reaction
        G_temp = []
        flag_int = 0 # Keeps updating when a GIS is filled into the matrix
        flag_prod = 0 # Keeps updating when a GP is filled into the matrix
        for j in range(spec_no):
            if j in interm_index: # checks if intermediate
                G_temp.append(np.abs(st_vec[j])*Gib['GIS'][flag_int])
                flag_int += 1
            elif j in prod_index: # checks if product
                G_temp.append(np.abs(st_vec[j])*Gib['GP'][flag_prod])
                flag_prod += 1
            else: # when the species has no free energy
                G_temp.append(0.0)
        G_full.append(G_temp)
    G_full = np.array(G_full) 

    for i in range(rno):
        # For forward reaction
        G_int_sum = sum([G_full[i][j] for j in range(len(G_full[i])) if st_mat[i][j] < 0]) 
        # the above line checks for those energies which are on the reactant side and sums them up
        G_forw = Gib['GTS'][i] - G_int_sum # GTS  -sum(GIS)
        
        k_ent[i,0] = kd(G_forw)
        # For reverse reaction
        G_int_sum = sum([G_full[i][j] for j in range(len(G_full[i])) if st_mat[i][j] > 0]) 
        # the above line checks for those energies which are on the product side and sums them up
        G_rev = Gib['GTS'][i] - G_int_sum   
        k_ent[i,1] = kd(G_rev)

    return k_ent

def solve_theta(Theta, Gibbs):
    "This function computes the mole balances for each intermediate and will be used"
    "for solving the steady state values of coverages"
    no_k = np.zeros(interm_no)
    Theta_s = 1 - sum(Theta) # converage of site
    k_arr = rate_constant(Gibbs) # array of rate constants
    dISdt = np.zeros(interm_no) # creates an array to calculate the mole balances for each intermediate
    tick = 0 # keeps track of the number of intermediates
    data_list = [] # contains the pressures and coverages of each species.
    # It stores the coverages for interm. and products, and pressures for other gases/reactants.
    for i in range(spec_no):
        if i in interm_index:
            data_list.append(Theta[tick])
            tick += 1
        else:
            data_list.append(dreact[i][1]) # Stores the pressure of that particular species
    data_list.append(Theta_s)
    data_list = np.array(data_list)
#     print(data_list)
   
    for i in range(interm_no):
#         print(i,'---------------------')
        # i th intermediate
        for j in range(rno):
            # for each reaction
            st_vec = st_mat[j] # Stoich vector of that reaction

            neg = np.where(st_vec<0) # Checking for indices where stoich is negative
            pos = np.where(st_vec>0) # Checking for indices where stoich is negative
            zero = np.where(st_vec==0) # Checking for indices where stoich is zero

            if st_vec[interm_index[i]] == 0:
                # if the stoich. of the intermediate is 0 for that particular reaction 
                # then it has nothing to do with the mole balance and hence zero is added.
                dISdt[i] += 0
            
            if st_vec[interm_index[i]] > 0:
                # if the stoich. of the intermediate is positve for that particular reaction 
                # then the forward rate constant is multiplied with the product of data_list^(st_vec) 
                # for those that are negative. And the converse for the reverse reaction
                prodd = 1
                prodd = np.prod([data_list[index]**np.abs(st_vec[index]) for index in neg])
                print(prodd,'IS pos : reactants neg')
                no_k[i] += prodd
                print(no_k[i])
                dISdt[i] += (k_arr[j][0]*prodd) # forward reaction 
                prodd = np.prod([data_list[index]**np.abs(st_vec[index]) for index in pos])
                dISdt[i] += (-k_arr[j][1]*prodd) # reverse reaction
                print(prodd,'IS pos : reactants pos')
                no_k[i] += -prodd
                print(no_k[i])
                
            if st_vec[interm_index[i]] < 0:
                # this is the same as the previous condition, only the fact that the intermediate is 
                # on the reactant side (in the equation)
                prodd = 1
                prodd = np.prod([data_list[index]**np.abs(st_vec[index]) for index in pos])
                print(prodd,'IS neg : reactants pos')
                no_k[i] += prodd
                print(no_k[i])
                dISdt[i] += (k_arr[j][1]*prodd) # forward reaction 
                prodd = np.prod([data_list[index]**np.abs(st_vec[index]) for index in neg])
                dISdt[i] += (-k_arr[j][0]*prodd) # reverse reaction
                print(prodd,'IS neg : reactants neg')
                no_k[i] += -prodd
                print(no_k[i])

    return dISdt

Theta_guess = np.array(Theta_guess)
# print(Theta_guess)
print(solve_theta(Theta_guess, G_dict))
# kkkk = rate_constant(G_dict)
# print(kkkk[0,0])
# print(kkkk[0,1])
# print(kkkk[1,0])
# print(kkkk[1,1])
# print(kkkk[2,0])
# print(kkkk[2,1])

0.30000000000000004 IS pos : reactants neg
0.30000000000000004
0.2 IS pos : reactants pos
0.10000000000000003
0.0 IS neg : reactants pos
0.10000000000000003
0.1 IS neg : reactants neg
2.7755575615628914e-17
0.9000000000000002 IS pos : reactants neg
0.9000000000000002
0.25 IS pos : reactants pos
0.6500000000000002
0.0 IS neg : reactants pos
0.6500000000000002
0.1 IS neg : reactants neg
0.5500000000000003
[1.27048146e+09 3.43955100e+12]


In [46]:
[ 1.00169672e+12 ,-2.81033969e+12]

[1001696720000.0, -2810339690000.0]

In [11]:
def lnrate(Gi):
    "This function computes the log of the rate based on the free energies that it takes"
    "Finite Differences and automatic differentiation used to compute the DRC later."
    G_list = Gi.tolist() # converting from array to a list

    seclst = [len(GTS), len(GIS), len(GP)] # splits the list
    # the following few lines are just to convert the list to a diction. which followed the
    # previous format.
    it = iter(G_list)
    sliced = [list(islice(it, 0, i)) for i in seclst]
    G_char = ['GTS','GIS','GP']
    Gib1 = {G_char[i] : np.array(sliced[i]) for i in range(len(sliced))}


    k_rate = rate_constant(Gib1) # array of rate constants
    thet = fsolve(solve_theta,Theta_guess, args = (Gib1,)) # gives coverages for all intermediates
    thet_s = 1 - sum(thet) # coverage of site.

    tick = 0 # keeps track of the number of intermediates
    
    data_list = []
    for i in range(spec_no):
        if i in interm_index:
            data_list.append(thet[tick]) # append the new coverages into the data_list
            tick += 1
        else:
            data_list.append(dreact[i][1]) # Stores the pressure of that particular species
    data_list.append(thet_s)
    data_list = np.array(data_list)
    
#     print(data_list)
    
    # for the particular reaction for which the rate will be calculated (input by user)
    st_vec = st_mat[rate_rno] # Stoich vector of that reaction
    
    neg = np.where(st_vec<0) # Checking for indices where stoich is negative
    pos = np.where(st_vec>0) # Checking for indices where stoich is negative
    zero = np.where(st_vec==0) # Checking for indices where stoich is zero

    product = np.prod([data_list[index]**np.abs(st_vec[index]) for index in neg])

    r_forw = k_rate[rate_rno][0] * product

    product = np.prod([data_list[index]**np.abs(st_vec[index]) for index in pos])

    r_rev = k_rate[rate_rno][1] * product

    r = r_forw - r_rev
    ret = np.log(r)
#     if math.isnan(ret): # there were cases when the log of the rate were nans (r was negative)
#         # need to check this properly
#         ret = np.log(-r)

    return ret # log of rate is returned

# print(lnrate(G_array))

In [12]:
rate_rno = input("Enter the reaction number that determines the rate :")
rate_rno = int(rate_rno) - 1

# Example input --------------------------------------
# Enter the reaction number that determines the rate :2
# ----------------------------------------------------

Enter the reaction number that determines the rate :2


In [13]:
"DRC calculation"

# The part below converts the dictionary to a 1D array since 
#nd.Gradient requires the arguments to not be a dictionary 
G_lst = chain.from_iterable(G_dict.values())
G_array = np.array(list(G_lst)) # Converting the dictionary to 1D array
G_list = G_array.tolist()


# print(solve_theta(Theta_guess, G_dict))

# Computing the term dln(r)/dG using finite differences 
dlndG_s = nd.Gradient(lnrate)
dlndGi = dlndG_s(G_array)



DRCi = -dlndGi.T * R * T
for i in range(len(DRCi)):
    print(f'Steady state DRC for {G_char[i]}: {DRCi[i]:1.4f}')


0.30000000000000004 IS pos : reactants neg
0.30000000000000004
0.2 IS pos : reactants pos
0.10000000000000003
0.0 IS neg : reactants pos
0.10000000000000003
0.1 IS neg : reactants neg
2.7755575615628914e-17
0.9000000000000002 IS pos : reactants neg
0.9000000000000002
0.25 IS pos : reactants pos
0.6500000000000002
0.0 IS neg : reactants pos
0.6500000000000002
0.1 IS neg : reactants neg
0.5500000000000003
0.30000000000000004 IS pos : reactants neg
0.30000000000000004
0.2 IS pos : reactants pos
0.10000000000000003
0.0 IS neg : reactants pos
0.10000000000000003
0.1 IS neg : reactants neg
2.7755575615628914e-17
0.9000000000000002 IS pos : reactants neg
0.9000000000000002
0.25 IS pos : reactants pos
0.6500000000000002
0.0 IS neg : reactants pos
0.6500000000000002
0.1 IS neg : reactants neg
0.5500000000000003
0.30000000000000004 IS pos : reactants neg
0.30000000000000004
0.2 IS pos : reactants pos
0.10000000000000003
0.0 IS neg : reactants pos
0.10000000000000003
0.1 IS neg : reactants neg
2.

0.9000000000000002 IS pos : reactants neg
0.9000000000000002
0.25 IS pos : reactants pos
0.6500000000000002
0.0 IS neg : reactants pos
0.6500000000000002
0.1 IS neg : reactants neg
0.5500000000000003
0.30000000000000004 IS pos : reactants neg
0.30000000000000004
0.2 IS pos : reactants pos
0.10000000000000003
0.0 IS neg : reactants pos
0.10000000000000003
0.1 IS neg : reactants neg
2.7755575615628914e-17
0.9000000000000002 IS pos : reactants neg
0.9000000000000002
0.25 IS pos : reactants pos
0.6500000000000002
0.0 IS neg : reactants pos
0.6500000000000002
0.1 IS neg : reactants neg
0.5500000000000003
0.29999999701976776 IS pos : reactants neg
0.29999999701976776
0.20000000298023224 IS pos : reactants pos
0.09999999403953552
0.0 IS neg : reactants pos
0.09999999403953552
0.10000000149011612 IS neg : reactants neg
-7.450580596923828e-09
0.8999999821186067 IS pos : reactants neg
0.8999999821186067
0.25 IS pos : reactants pos
0.6499999821186067
0.0 IS neg : reactants pos
0.6499999821186067


0.5499999806284905
0.29999999254941945 IS pos : reactants neg
0.29999999254941945
0.2 IS pos : reactants pos
0.09999999254941944
0.0 IS neg : reactants pos
0.09999999254941944
0.10000000149011612 IS neg : reactants neg
-8.940696683001903e-09
0.8999999552965172 IS pos : reactants neg
0.8999999552965172
0.25000000745058065 IS pos : reactants pos
0.6499999478459366
0.0 IS neg : reactants pos
0.6499999478459366
0.10000000149011612 IS neg : reactants neg
0.5499999463558205
0.22985114773614124 IS pos : reactants neg
0.22985114773614124
0.13537519469199533 IS pos : reactants pos
0.09447595304414591
0.0 IS neg : reactants pos
0.09447595304414591
0.08593260747914098 IS neg : reactants neg
0.008543345565004926
0.5283155011562143 IS pos : reactants neg
0.5283155011562143
0.40293759634716136 IS pos : reactants pos
0.12537790480905292
0.0 IS neg : reactants pos
0.12537790480905292
0.08593260747914098 IS neg : reactants neg
0.039445297329911935
0.22405969169646167 IS pos : reactants neg
0.2240596916

-0.00028471900346328793
0.5020039725096639 IS pos : reactants neg
0.5020039725096639
0.4084184924017567 IS pos : reactants pos
0.09358548010790718
0.0 IS neg : reactants pos
0.09358548010790718
0.08746990264735081 IS neg : reactants neg
0.006115577460556368
0.22277622303658018 IS pos : reactants neg
0.22277622303658018
0.13567318036406878 IS pos : reactants pos
0.0871030426725114
0.0 IS neg : reactants pos
0.0871030426725114
0.08704120980509969 IS neg : reactants neg
6.183286741170968e-05
0.4962924555044412 IS pos : reactants neg
0.4962924555044412
0.4115871679969833 IS pos : reactants pos
0.08470528750745787
0.0 IS neg : reactants pos
0.08470528750745787
0.08704120980509969 IS neg : reactants neg
-0.002335922297641818
0.22307059504118454 IS pos : reactants neg
0.22307059504118454
0.1360447287675086 IS pos : reactants pos
0.08702586627367595
0.0 IS neg : reactants pos
0.08702586627367595
0.08718898194369891 IS neg : reactants neg
-0.00016311567002295912
0.49760490372028143 IS pos : rea

0.08719306040641796 IS neg : reactants neg
8.158959418740064e-05
0.22312983719245028 IS pos : reactants neg
0.22312983719245028
0.1360633282452044 IS pos : reactants pos
0.08706650894724588
0.0 IS neg : reactants pos
0.08706650894724588
0.08719031067282679 IS neg : reactants neg
-0.00012380172558090763
0.4978692424552936 IS pos : reactants neg
0.4978692424552936
0.410633399221813 IS pos : reactants pos
0.0872358432334806
0.0 IS neg : reactants pos
0.0872358432334806
0.08719031067282679 IS neg : reactants neg
4.5532560653815946e-05
0.2231298514635337 IS pos : reactants neg
0.2231298514635337
0.13606334145158047 IS pos : reactants pos
0.08706651001195323
0.0 IS neg : reactants pos
0.08706651001195323
0.08719031539688787 IS neg : reactants neg
-0.00012380538493464432
0.49786930614138614 IS pos : reactants neg
0.49786930614138614
0.41063336400632605 IS pos : reactants pos
0.08723594213506008
0.0 IS neg : reactants pos
0.08723594213506008
0.08719031539688787 IS neg : reactants neg
4.5626738

...

0.1 IS neg : reactants neg
2.7755575615628914e-17
0.9000000000000002 IS pos : reactants neg
0.9000000000000002
0.25 IS pos : reactants pos
0.6500000000000002
0.0 IS neg : reactants pos
0.6500000000000002
0.1 IS neg : reactants neg
0.5500000000000003
0.30000000000000004 IS pos : reactants neg
0.30000000000000004
0.2 IS pos : reactants pos
0.10000000000000003
0.0 IS neg : reactants pos
0.10000000000000003
0.1 IS neg : reactants neg
2.7755575615628914e-17
0.9000000000000002 IS pos : reactants neg
0.9000000000000002
0.25 IS pos : reactants pos
0.6500000000000002
0.0 IS neg : reactants pos
0.6500000000000002
0.1 IS neg : reactants neg
0.5500000000000003
0.29999999701976776 IS pos : reactants neg
0.29999999701976776
0.20000000298023224 IS pos : reactants pos
0.09999999403953552
0.0 IS neg : reactants pos
0.09999999403953552
0.10000000149011612 IS neg : reactants neg
-7.450580596923828e-09
0.8999999821186067 IS pos : reactants neg
0.8999999821186067
0.25 IS pos : reactants pos
0.6499999821186

0.10000000149011612 IS neg : reactants neg
0.5499999806284905
0.29999999254941945 IS pos : reactants neg
0.29999999254941945
0.2 IS pos : reactants pos
0.09999999254941944
0.0 IS neg : reactants pos
0.09999999254941944
0.10000000149011612 IS neg : reactants neg
-8.940696683001903e-09
0.8999999552965172 IS pos : reactants neg
0.8999999552965172
0.25000000745058065 IS pos : reactants pos
0.6499999478459366
0.0 IS neg : reactants pos
0.6499999478459366
0.10000000149011612 IS neg : reactants neg
0.5499999463558205
0.2298511495128286 IS pos : reactants neg
0.2298511495128286
0.13537518918486263 IS pos : reactants pos
0.09447596032796596
0.0 IS neg : reactants pos
0.09447596032796596
0.08593260448836798 IS neg : reactants neg
0.008543355839597977
0.5283155093236868 IS pos : reactants neg
0.5283155093236868
0.4029376010831383 IS pos : reactants pos
0.12537790824054845
0.0 IS neg : reactants pos
0.12537790824054845
0.08593260448836798 IS neg : reactants neg
0.03944530375218047
0.22405969373421

0.08718650955453691
0.08747115116003494 IS neg : reactants neg
-0.00028464160549802886
0.5020274544311345 IS pos : reactants neg
0.5020274544311345
0.4084067917404637 IS pos : reactants pos
0.09362066269067082
0.0 IS neg : reactants pos
0.09362066269067082
0.08747115116003494 IS neg : reactants neg
0.006149511530635879
0.22278172393784712 IS pos : reactants neg
0.22278172393784712
0.13567739356041844 IS pos : reactants pos
0.08710433037742868
0.0 IS neg : reactants pos
0.08710433037742868
0.08704259480028599 IS neg : reactants neg
6.173557714268807e-05
0.49631696520719126 IS pos : reactants neg
0.49631696520719126
0.4115747039211043 IS pos : reactants pos
0.08474226128608697
0.0 IS neg : reactants pos
0.08474226128608697
0.08704259480028599 IS neg : reactants neg
-0.0023003335141990183
0.22307599566005165 IS pos : reactants neg
0.22307599566005165
0.13604880725369714 IS pos : reactants pos
0.08702718840635451
0.0 IS neg : reactants pos
0.08702718840635451
0.08719030616206255 IS neg : r

0.08690797572889436
0.08719030600025035 IS neg : reactants neg
-0.00028233027135599054
0.2230707258014124 IS pos : reactants neg
0.2230707258014124
0.13604190330314947 IS pos : reactants pos
0.08702882249826294
0.0 IS neg : reactants pos
0.08702882249826294
0.08718753773956689 IS neg : reactants neg
-0.00015871524130395276
0.4976054870956892 IS pos : reactants neg
0.4976054870956892
0.4107366221732669 IS pos : reactants pos
0.0868688649224223
0.0 IS neg : reactants pos
0.0868688649224223
0.08718753773956689 IS neg : reactants neg
-0.00031867281714459406
0.2230707402555243 IS pos : reactants neg
0.2230707402555243
0.1360419166710769 IS pos : reactants pos
0.08702882358444741
0.0 IS neg : reactants pos
0.08702882358444741
0.0871875425219392 IS neg : reactants neg
-0.0001587189374917808
0.49760555158147596 IS pos : reactants neg
0.49760555158147596
0.4107365865116804 IS pos : reactants pos
0.08686896506979558
0.0 IS neg : reactants pos
0.08686896506979558
0.0871875425219392 IS neg : react

0.0 IS neg : reactants pos
0.08702882829174047
0.08718754315033755 IS neg : reactants neg
-0.00015871485859708656
0.4976055879311827 IS pos : reactants neg
0.4976055879311827
0.410736571658662 IS pos : reactants pos
0.0868690162725207
0.0 IS neg : reactants pos
0.0868690162725207
0.08718754315033755 IS neg : reactants neg
-0.0003185268778168604
0.30000000000000004 IS pos : reactants neg
0.30000000000000004
0.2 IS pos : reactants pos
0.10000000000000003
0.0 IS neg : reactants pos
0.10000000000000003
0.1 IS neg : reactants neg
2.7755575615628914e-17
0.9000000000000002 IS pos : reactants neg
0.9000000000000002
0.25 IS pos : reactants pos
0.6500000000000002
0.0 IS neg : reactants pos
0.6500000000000002
0.1 IS neg : reactants neg
0.5500000000000003
0.30000000000000004 IS pos : reactants neg
0.30000000000000004
0.2 IS pos : reactants pos
0.10000000000000003
0.0 IS neg : reactants pos
0.10000000000000003
0.1 IS neg : reactants neg
2.7755575615628914e-17
0.9000000000000002 IS pos : reactants n

In [59]:
0.3158 + 0.7221 - 0.0379

1.0

The rate constants are accurate but the mole balances are not. This lead to the change in DRCs as well. The expected DRCs are supposed to be: <br>
GTS1 :-0.0000 <br>
GTS2 :0.3158<br>
GTS3 :0.7221<br>
GAS :0.2638<br>
GBS :-0.9007<br>
GABS : -0.0000<br>

Expected mole balances: [ 1.00169672e+12 -2.81033969e+12]